# Problema

Nuestro interés es generar un modelo grande de lenguaje que pueda responder preguntas en base a las leyes laborales existentes. Para resolver esto utilizaremos el enfoque de RAG para LLMs.

La abreviatura de "Retrieval-Augmented Generation" en español es **RAG**. Se utiliza comúnmente en el ámbito de la inteligencia artificial y el procesamiento del lenguaje natural para referirse a la técnica de generación aumentada por recuperación. Esta técnica combina un modelo de lenguaje grande (LLM) con un componente de recuperación de información para mejorar la precisión y confiabilidad de las respuestas generadas por el LLM.

<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Frag-framework.81dc2cdc.png&w=3840&q=75" height="700"> Fuente: promptingguide.ai

RAG toma como entrada un texto y recupera un conjunto de documentos relevantes o de apoyo dada una fuente, que en nuestro caso son los textos de las leyes laborales. Los documentos se concatenan como contexto con el texto original y se alimentan al generador de texto, que produce la salida final. Esto hace que RAG se adapte a situaciones en las que los hechos pueden evolucionar con el tiempo. Esto es muy útil ya que el conocimiento paramétrico de los LLM es estático. RAG permite a los modelos de lenguaje evitar el reentrenamiento, permitiendo el acceso a la información más reciente para generar resultados confiables a través de la generación basada en recuperación.



<img src="https://docs.aws.amazon.com/images/sagemaker/latest/dg/images/jumpstart/jumpstart-fm-rag.jpg" height="700">
Fuente: Amazon

En el gráfico anterior podemos observar el paso a paso de como se resuelve una consulta al modelo.
- 1. Se recibe un prompt + query
- 2. El query se utiliza para realizar una consulta en una base de datos de embeddings de nuestros documentos
- 3. Se recuperan los textos relevantes que aporten a la consulta, ese es el con texto.
- 4. se enriquece el prompt sumándole el contexto. Esa consulta es la que pasa al LLM.
- 5. El LLM genera una respuesta.


Para poder realizar todo lo anterior es importante realizar un paso previo a poner cualquier modelo en producción: construir nuestra base de datos, eso lo tratamos en el apartado **Base de datos**.

En el apartado **Retriever - Procesar consulta** implementaremos los necesario para recibir la consulta, consultar los documentos relevantes a la base de datos.

En la sección **Reader - LLM** configuraremos el LLM a utilizar para funcionar recibiendo información del contexto sobre el cual debe responder.




<img src="https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/RAG_workflow.png" height="700">
Fuente: HuggingFace


# Configuraciones

## Librerías

Instalación de librerias a usar en este notebook

In [ ]:
!pip install -q langchain_community langchain chromadb sentence-transformers

In [ ]:
!pip install ragas
!pip install colab-xterm

In [ ]:
! pip install langchain_community

# Base de datos






## Origen de los datos

Tomando de referencia la página oficial del gobierno argentino sobre “Trabajo, Empleo y Seguridad Social” (https://www.argentina.gob.ar/trabajo/buscastrabajo/conocetusderechos/marcolegal), usaremos como fuente de datos las siguientes leyes laborales:

* https://www.argentina.gob.ar/normativa/nacional/ley-20744-25552/actualizacion
* https://www.argentina.gob.ar/normativa/nacional/ley-24013-412/actualizacion
* https://www.argentina.gob.ar/normativa/nacional/ley-24557-27971/actualizacion
* https://www.argentina.gob.ar/normativa/nacional/ley-11544-63368/actualizacion

## Carga de datos



Descarga de datos:  
Luego de esta prueba preliminar procedemos a realizar un script para automatizar la descarga del texto de las leyes y tener los datos descargados como archivos .txt para manipularlos luego.

In [ ]:
# los enlaces de interés
links = [
"https://www.argentina.gob.ar/normativa/nacional/ley-20744-25552/actualizacion",
"https://www.argentina.gob.ar/normativa/nacional/ley-24013-412/actualizacion",
"https://www.argentina.gob.ar/normativa/nacional/ley-24557-27971/actualizacion",
"https://www.argentina.gob.ar/normativa/nacional/ley-11544-63368/actualizacion"
]

Descarga de cada uno de los enlaces y se guarda en una lista de objetos Document

In [ ]:
from langchain_community.document_loaders import AsyncHtmlLoader

loader = AsyncHtmlLoader(links)
docs = loader.load()

Fetching pages: 100%|##########| 4/4 [00:10<00:00,  2.72s/it]


## Preprocesamiento

Utilizamos BeautifulSoup para que de cada sitio web extraiga el texto entre los tags article y nos quede en un objeto tipo Document.

In [ ]:
from langchain_community.document_transformers import BeautifulSoupTransformer
# Transform
bs_transformer = BeautifulSoupTransformer()
docs_transformed = bs_transformer.transform_documents(docs, tags_to_extract=["article"])

Observamos que creó 4 documentos

In [ ]:
len(docs_transformed)

4

Vemos un poco de contenido de alguno

In [ ]:
docs_transformed[0].page_content

"REGIMEN DE CONTRATO DE TRABAJO  LEY N° 20.744 - TEXTO ORDENADO POR DECRETO 390/1976  Bs. As., 13/5/1976  Ver Antecedentes Normativos  LEY DE CONTRATO DE TRABAJO.  TITULO I  Disposiciones Generales  Artículo 1° — Fuentes de regulación.  El contrato de trabajo y la relación de trabajo se rige:  a) Por esta ley.  b) Por las leyes y estatutos profesionales.  c) Por las convenciones colectivas o laudos con fuerza de tales.  d) Por la voluntad de las partes.  e) Por los usos y costumbres.  Art. 2° — Ambito de aplicación.  La vigencia de esta Ley quedará condicionada a que la aplicación de sus disposiciones resulte compatible con la naturaleza y modalidades de la actividad de que se trate y con el específico régimen jurídico a que se halle sujeta. Las disposiciones de esta Ley no serán aplicables:  a. A los dependientes de la Administración Pública Nacional, Provincial, de la Ciudad Autónoma de Buenos Aires o Municipal, excepto que por acto expreso se los incluya en la misma o en el régimen 

In [ ]:
docs_transformed[0].metadata

{'source': 'https://www.argentina.gob.ar/normativa/nacional/ley-20744-25552/actualizacion',
 'title': 'Texto actualizado | Argentina.gob.ar',
 'description': 'Portal oficial del Estado argentino. Conocé cómo hacer trámites en organismos públicos, tramitalos en línea y recibí servicios digitales y beneficios.',
 'language': 'es'}

## Modificación de la metadata de los documentos

In [ ]:
titles = [
"REGIMEN DE CONTRATO DE TRABAJO  LEY N° 20.744",
"EMPLEO Ley Nº 24.013",
"RIESGOS DEL TRABAJO Ley N° 24.557",
"JORNADA DE TRABAJOLey 11.544"
]

In [ ]:
from langchain_core.documents import Document
documents = []
for i in range(len(docs_transformed)):
  metadata = {"title": titles[i], "source": links[i]}
  d = Document(metadata=metadata, page_content = docs_transformed[i].page_content)
  documents.append(d)

In [ ]:
documents[0].metadata

{'title': 'REGIMEN DE CONTRATO DE TRABAJO  LEY N° 20.744',
 'source': 'https://www.argentina.gob.ar/normativa/nacional/ley-20744-25552/actualizacion'}

## División del texto en secciones (chunks)

In [ ]:
# taken from https://raw.githubusercontent.com/pixegami/rag-tutorial-v2/main/populate_database.py
from langchain_text_splitters import RecursiveCharacterTextSplitter

def split_documents(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=80,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_documents(documents)

In [ ]:
chunks = split_documents(documents)
chunks

In [ ]:
# taken from https://raw.githubusercontent.com/pixegami/rag-tutorial-v2/main/populate_database.py
def calculate_chunk_ids(chunks):

    # This will create IDs like "data/violencia.pdf:16:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        current_page_id = f"{source}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [ ]:
# Calculate IDs for each text chunk
chunks_with_ids = calculate_chunk_ids(chunks)

# Embeddings

## Función get_embedding_function()

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

def get_embedding_function(embedding_name):
  # Create a dictionary with model configuration options, specifying to use the CPU for computations
  model_kwargs = {'device':'cuda'}

  # Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
  encode_kwargs = {'normalize_embeddings': True}

  # Initialize an instance of HuggingFaceEmbeddings with the specified parameters
  embeddings = HuggingFaceEmbeddings(
      model_name=embedding_name,     # Provide the pre-trained model's path
      model_kwargs=model_kwargs, # Pass the model configuration options
      encode_kwargs=encode_kwargs # Pass the encoding options
  )
  return embeddings

## Creacion de bases de datos vectorial

In [ ]:
from langchain.vectorstores.chroma import Chroma

# create and persist the database
def create_database(db_path=None, texts = None, embeddings=None):

  if db_path is None or texts is None or embeddings is None:
    print("ERROR pasar los parametros necesarios")
    return

  db = Chroma( persist_directory=db_path, embedding_function=embeddings)

  db.add_documents(texts)
  db.persist()

  return db

## Ollama

Descargamos los modelos LLM via Ollama



In [ ]:
# Load the extension
%load_ext colabxterm

In [ ]:
# NO EJECUTAR EN ESTA CELDA
# copy & paste en la xterm terminal de abajo
# curl -fsSL https://ollama.com/install.sh | sh
# ollama serve &
# ollama pull llama3-chatqa

In [ ]:
%xterm height=300

Launching Xterm...

<IPython.core.display.Javascript object>

Compruebo el modelo llama3-chatqa

In [ ]:
from langchain_community.llms.ollama import Ollama
critic_llm = Ollama(model="llama3-chatqa")
response_text = critic_llm.invoke("¿Es legal que mi jefe me despida sin avisarme antes? ")
response_text

' Como asistente de IA no puedo emitir un juicio de derecho. Sin embargo, puedo informarte sobre lo que establece la legislación española respecto a este tema. Según la ley española, el empleador debe notificar al trabajador con una antelación mínima de 15 días y mediante justificación si se produce la despedida por causa echada o la extinción del contrato por motivos económicos, técnicos o organizativos. Por tanto, para que sea legal debería haber un aviso previo con el motivo de la despido.'

## Dataset para evaluar

In [ ]:
questions = ["¿Cuáles son las garantías para el pago de las remuneraciones al trabajador en caso de concurrencia de acreedores?",
             "¿Cuánto es el máximo tiempo de suspensión por fuerza mayor?",
             "¿Cuál es la duración máxima de la suspensión por causas disciplinarias o falta de trabajo no imputables al empleador?",
             "¿Mi jefe puede despedirme sin aviso previo?",
             "¿Con cuanta anticipación hay que informarle el despido a un trabajador si aún está en el período de prueba?"
]

In [ ]:
ground_truth = ["El trabajador tendrá derecho a ser pagado, con preferencia a otros acreedores del empleador",
                "Las suspensiones por fuerza mayor debidamente comprobadas podrán extenderse hasta un plazo máximo de setenta y cinco 75 días en el término de un 1 año",
                "30 días en un año, contados a partir de la primera suspensión.",
                "Tienes derecho que te avisen con antelación del cese de tu contrato de trabajo. Caso contrario, la empresa debe abonarte junto a tu indemnización el preaviso.",
                "15 (quince) días"
                ]

#"por el empleador, de QUINCE (15) días cuando el trabajador se encontrare en período de prueba; de UN (1) mes cuando el trabajador tuviese una antigüedad en el empleo que no exceda de CINCO (5) años y de DOS (2) meses cuando fuere superior"

## Embeddings ll-MiniLM-L6-v2

In [ ]:
EMBED_MODEL = ["all-MiniLM-L6-v2",'hiiamsid/sentence_similarity_spanish_es','jinaai/jina-embeddings-v2-base-es']

In [ ]:
db = create_database(db_path="chroma1", texts = chunks, embeddings=get_embedding_function(EMBED_MODEL[0]))

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import (RunnableParallel, RunnablePassthrough )
from langchain.schema.output_parser import StrOutputParser
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

In [ ]:
PROMPT_TEMPLATE = """
Responda a la pregunta basandose únicamente en el siguiente contexto:

{context}

---

Responda en español la pregunta, basándose en el contexto brindado antes: {question}
"""

In [ ]:
MODEL="llama3-chatqa"

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model = MODEL)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retrieval = RunnableParallel(
    {"context": retriever,  "question": RunnablePassthrough()}
)

chain = retrieval | prompt | llm | StrOutputParser()

In [ ]:
answers  = []
contexts = []

In [ ]:
# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(chain.invoke(question))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

Vemos las respuestas

In [ ]:
answers

[' Las remuneraciones que se fijen por las convenciones colectivas deberán expresarse, en su totalidad, en dinero. El empleador no podrá imputar los pagos en especies a más del veinte (20) por ciento del total de la remuneración.',
 ' 75 días',
 ' Según el artículo 219 de la ley N°20.744, la duración máxima de la suspensión por causas disciplinarias o falta de trabajo no imputables al empleador es indefinida.',
 ' No, según los artículos 19 y 20 de la Ley N° 20.744 (t.o. 1976), un empleador debe darle a su trabajador un plazo de preaviso cuando el contrato termina.',
 'De acuerdo con lo establecido en la Ley N°20.744, durante el periodo de prueba se deben dar al trabajador un preaviso con una anticipación de QUINCE (15) días para el caso de que no exista una fecha específica acordada entre las partes, o en su defecto, una anticipación equivalente a SEIS (6) meses para los contratos cuyo término sea mayor a DOS (2) años.']

In [ ]:
# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 5
})

In [ ]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

super_llama = Ollama(model = "llama3")

In [ ]:
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

result = evaluate(
    dataset=dataset,
    metrics=[context_precision, context_recall, faithfulness, answer_relevancy],
    llm=super_llama,
    embeddings=get_embedding_function(EMBED_MODEL[0])
)

df = result.to_pandas()

In [ ]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall
0,¿Cuáles son las garantías para el pago de las ...,Las remuneraciones que se fijen por las conve...,[Las remuneraciones que se fijen por las conve...,"El trabajador tendrá derecho a ser pagado, con...",1.000000,1.0
1,¿Cuánto es el máximo tiempo de suspensión por ...,75 días,[hasta un plazo máximo de setenta y cinco (75)...,Las suspensiones por fuerza mayor debidamente ...,1.000000,1.0
2,¿Cuál es la duración máxima de la suspensión p...,"Según el artículo 219 de la ley N°20.744, la ...",[serán valorizadas adecuadamente. La suspensi...,"30 días en un año, contados a partir de la pri...",1.000000,0.8
3,¿Mi jefe puede despedirme sin aviso previo?,"No, según los artículos 19 y 20 de la Ley N° ...","[vivienda digna, educación, vestuario, asisten...",Tienes derecho que te avisen con antelación de...,0.500000,0.0
4,¿Con cuanta anticipación hay que informarle el...,De acuerdo con lo establecido en la Ley N°20.7...,[empleador ha renunciado al período de prueba....,15 (quince) días,0.638889,0.5


In [ ]:
# guardar df como archivo .csv
df.to_csv(str(EMBED_MODEL[0]+'.csv'),sep=",", encoding='utf-8')

## Embeddings 'hiiamsid/sentence_similarity_spanish_es'

In [ ]:
embeddings = get_embedding_function(EMBED_MODEL[1])

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
db = create_database(db_path="chroma2", texts = chunks, embeddings=embeddings)

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import (RunnableParallel, RunnablePassthrough)
from langchain.schema.output_parser import StrOutputParser
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

In [ ]:
# models = ["tinyllama","llama2","llama3-chatqa"]
MODEL="llama3-chatqa"

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model = MODEL)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retrieval = RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

chain = retrieval | prompt | llm | StrOutputParser()

In [ ]:
answers  = []
contexts = []

In [ ]:
# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(chain.invoke(question))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

Vemos las respuestas

In [ ]:
answers

[' Las garantías que se establecen para el pago de las remuneraciones al trabajador en caso de concurrencia de acreedores incluyen el depósito de una cuota mensual equivalente a un día laboral o la constitución de una fianza por importe igual.',
 ' 75 días',
 ' 30 días',
 ' No, tu contrato laboral establece que tienes derecho a un preaviso en caso de despido.',
 ' 15 días']

In [ ]:
# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 5
})

In [ ]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

super_llama = Ollama(model = "llama3")

In [ ]:
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

result = evaluate(
    dataset=dataset,
    metrics=[context_precision, context_recall],
    llm=super_llama,
    embeddings=embeddings)

df = result.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall
0,¿Cuáles son las garantías para el pago de las ...,Las garantías que se establecen para el pago ...,[hechas con las formalidades del Art. 130 de e...,"El trabajador tendrá derecho a ser pagado, con...",0.805556,1.000000
1,¿Cuánto es el máximo tiempo de suspensión por ...,75 días,"[de trabajo no imputables al empleador, no pod...",Las suspensiones por fuerza mayor debidamente ...,1.000000,1.000000
2,¿Cuál es la duración máxima de la suspensión p...,30 días,"[de trabajo no imputable al empleador, a razon...","30 días en un año, contados a partir de la pri...",0.916667,0.250000
3,¿Mi jefe puede despedirme sin aviso previo?,"No, tu contrato laboral establece que tienes ...","[irrazonable de esa facultad, ni alteren modal...",Tienes derecho que te avisen con antelación de...,1.000000,0.666667
4,¿Con cuanta anticipación hay que informarle el...,15 días,[por antigüedad o despido. En los casos de des...,15 (quince) días,1.000000,0.666667


In [ ]:
# prompt: Mediante el marco de datos df: guardar df como csv
df.to_csv("df-embeb2.csv")

In [ ]:
str(EMBED_MODEL[1])

'hiiamsid/sentence_similarity_spanish_es'

## Embeddings 'jinaai/jina-embeddings-v2-base-es'

Probamos algunas consultas a la base de datos para observar los textos que regresa

In [ ]:
embeddings = get_embedding_function(EMBED_MODEL[2])

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/83.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/592k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
db = create_database(db_path="chroma3", texts = chunks, embeddings=embeddings)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import (RunnableParallel, RunnablePassthrough)
from langchain.schema.output_parser import StrOutputParser
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

In [ ]:
# models = ["tinyllama","llama2","llama3-chatqa"]
MODEL="llama3-chatqa"

In [ ]:
from langchain_community.llms import Ollama

llm = Ollama(model = MODEL)
prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retrieval = RunnableParallel({"context": retriever,  "question": RunnablePassthrough()})

chain = retrieval | prompt | llm | StrOutputParser()

In [ ]:
answers  = []
contexts = []

In [ ]:
# traversing each question and passing into the chain to get answer from the system
for question in questions:
    answers.append(chain.invoke(question))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])

Vemos las respuestas

In [ ]:
answers

[' Las empresas deberán estar constituidas exclusivamente como personas jurídicas y con objeto único. Sólo podrán mediar en la contratación de trabajadores bajo la modalidad de trabajo eventual.',
 'En Argentina, el máximo tiempo de suspensión por fuerza mayor establecido en las disposiciones laborales nacionales y reglamentarias es de hasta tres días. Sin embargo, esto puede variar dependiendo del convenio colectivo de trabajo aplicable y las condiciones específicas del contrato individual.',
 ' 6 meses',
 ' No puedes ser despedido sin previo aviso si no has cometido ningún tipo de falta que te haga merecedor a tal sanción.',
 ' Cuando un trabajador está en periodo de prueba y es despedido por parte del empleador se debe dar una anticipación del despido con un mínimo de treinta días.']

In [ ]:
# Preparing the dataset
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truth
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 5
})

In [ ]:
from langchain_core.language_models import BaseLanguageModel
from langchain_core.embeddings import Embeddings

super_llama = Ollama(model = "llama3")

In [ ]:
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, context_recall, context_precision)

result = evaluate(
    dataset=dataset,
    metrics=[context_precision, context_recall],
    llm=super_llama,
    embeddings=embeddings)

df = result.to_pandas()

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
df

,question,answer,contexts,ground_truth,context_precision,context_recall
0,¿Cuáles son las garantías para el pago de las ...,Las empresas deberán estar constituidas exclu...,[ARTICULO 77. — Las empresas de servicios even...,"El trabajador tendrá derecho a ser pagado, con...",1.000000,1.000000
1,¿Cuánto es el máximo tiempo de suspensión por ...,"En Argentina, el máximo tiempo de suspensión p...",[procederá cuando lo establezcan las disposici...,Las suspensiones por fuerza mayor debidamente ...,1.000000,0.000000
2,¿Cuál es la duración máxima de la suspensión p...,6 meses,[ante el empleador los accidentes y enfermedad...,"30 días en un año, contados a partir de la pri...",0.333333,1.000000
3,¿Mi jefe puede despedirme sin aviso previo?,No puedes ser despedido sin previo aviso si n...,[excederá de seis horas diarias o treinta y se...,Tienes derecho que te avisen con antelación de...,1.000000,0.666667
4,¿Con cuanta anticipación hay que informarle el...,Cuando un trabajador está en periodo de prueb...,"[la tarea de cada equipo, serán fijadas de tal...",15 (quince) días,0.000000,0.000000


In [ ]:
EMBED_MODEL[2]

'jinaai/jina-embeddings-v2-base-es'

In [ ]:
# guardar df como archivo .csv
df.to_csv('jina-embeb3.csv',sep=",", encoding='utf-8')

# Referencias

[1] https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

[2] https://huggingface.co/hiiamsid/sentence_similarity_spanish_es

[3] https://huggingface.co/jinaai/jina-embeddings-v2-base-es
